In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
from random import randint

In [2]:
headers = {"Accept-Language": "en-US,en;q=0.5"}

In [3]:
movie_name = []
year = []
time = []
rating = []
certificate = []
metascore = []
votes = []
gross = []
genre = []
description = []
director = []
cast = []

In [4]:
pages = np.arange(1,10000,100)

In [5]:
for page in pages:
    
    page = requests.get('https://www.imdb.com/search/title/?title_type=feature,tv_movie,short&count=100&start="+str(page)+"&ref_=adv_nxt')
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(1,2))
    
    for store in movie_data:
        
        name = store.h3.a.text
        movie_name.append(name)

        release_year = re.findall(r'\d+', store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text)[0] if store.h3.find('span', class_ = 'lister-item-year text-muted unbold') else ''
        year.append(release_year)

        runtime = re.findall(r'\d+', store.p.find('span', class_ = 'runtime').text)[0] if store.p.find('span', class_ = 'runtime') else ''
        time.append(runtime)

        rate = re.findall(r'\d+', store.find('div', class_ = 'inline-block ratings-imdb-rating').text)[0] if store.find('div', class_ = 'inline-block ratings-imdb-rating') else ''
        rating.append(rate)

        meta  = re.findall(r'\d+', store.find('span', class_ = 'metascore').text)[0] if store.find('span', class_ = 'metascore') else ''
        metascore.append(meta)
        
        value = store.find_all('span', attrs = {'name': 'nv'})
        
        vote = value[0].text.replace(',', '') if value else ''
        votes.append(vote)
    
        grosses = value[1].text if len(value) >1 else ''
        gross.append(grosses)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else ''
        description.append(description_)
        
        genres = store.p.find('span', class_ = 'genre').text.strip() if store.p.find('span', class_ = 'genre') else ''
        genre.append(genres)
        
        certificates = store.p.find('span', class_ = 'certificate').text if store.p.find('span', class_ = 'certificate') else ''
        certificate.append(certificates)
        
        casting = store.find("p", class_ = '')
        casting = casting.text.replace('\n', '').split('|')
        casting = [x.strip() for x in casting]
        casting = [casting[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
        director.append(casting[0])
        casting.pop(0)
        cast.append(', '.join(casting))

In [6]:
movie_DF = pd.DataFrame({'Movie': movie_name, 'Year of relase': year, 'Watchtime': time, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Genre': genre, 'Certified': certificate, 'Gross collection': gross, 'Description': description, 'Director': director, 'Cast': cast})

In [7]:
movie_DF

,Movie,Year of relase,Watchtime,Movie Rating,Metascore,Votes,Genre,Certified,Gross collection,Description,Director,Cast
0,The Menu,2022,107,7,71,175230,"Comedy, Horror, Thriller",15,,A young couple travels to a remote island to e...,Mark Mylod,"Ralph Fiennes, Anya Taylor-Joy, Nicholas Hoult..."
1,Avatar: The Way of Water,2022,192,7,67,255821,"Action, Adventure, Fantasy",11,,Jake Sully lives with his newfound family form...,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver..."
2,The Pale Blue Eye,2022,128,6,56,65990,"Crime, Horror, Mystery",16+,,A world-weary detective is hired to investigat...,Scott Cooper,"Christian Bale, Harry Melling, Simon McBurney,..."
3,Babylon,2022,189,7,60,24073,"Comedy, Drama, History",15,,A tale of outsized ambition and outrageous exc...,Damien Chazelle,"Brad Pitt, Margot Robbie, Jean Smart, Olivia W..."
4,M3GAN,2022,102,6,72,25449,"Horror, Sci-Fi, Thriller",15,,A robotics engineer at a toy company builds a ...,Gerard Johnstone,"Allison Williams, Violet McGraw, Ronny Chieng,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Ticket to Paradise,2022,104,6,50,37649,"Comedy, Romance",Btl,,A divorced couple teams up and travels to Bali...,Ol Parker,"George Clooney, Sean Lynch, Julia Roberts, Ari..."
9996,Evil Dead Rise,2023,97,,,,Horror,15,,A twisted tale of two estranged sisters whose ...,Lee Cronin,"Alyssa Sutherland, Lily Sullivan, Morgan Davie..."
9997,Love,2015,135,6,51,60496,"Drama, Romance",15,,Murphy is an American living in Paris who ente...,Gaspar Noé,"Aomi Muyock, Karl Glusman, Klara Kristin, Juan..."
9998,The Wolf of Wall Street,2013,180,8,75,1421269,"Biography, Comedy, Crime",15,$116.90M,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,"Leonardo DiCaprio, Jonah Hill, Margot Robbie, ..."
